In [3]:
import xarray

import sys
sys.path.append(sys.path[0] + '/..')
import utils._indexDefinitions as _index

In [2]:
def applyCriteria(indexDa, critDa):
    """This function applies the criteria to the indeces provided to determine if the events are negative or positive events.
    
    Then it sums events which are fire promoting or not-fire promoting.
    
    Should return two dataArrays with the count of fire Promoting and non fire Promoting modes for the models and years in the input data.
    
    """



    #enso and iod events are positive, sam events are negative
    indexNames=list(critDa.index.values)
    
    firePos=list(set(_index.firePos).intersection(indexNames))
    fireNeg=list(set(_index.fireNeg).intersection(indexNames))

    #events are greater than or less than the criteria
    posEvents=xarray.concat(
        (
            [
            (indexDa>critDa).sel(index=iNam)
            for iNam in indexNames
            ]
            #(indecesDa>criteriaDa).sel(index=['nino34','dmi']),
            #(indecesDa<-criteriaDa).sel(index='sam')
        ),
        'index'
    )

    negEvents=xarray.concat(
        (
            [
            (indexDa<-1*critDa).sel(index=iNam)
            for iNam in indexNames
            ]
        ),
        'index'
    ) 
    
    # a positive impact is a positive event with a positive impact, or a negative event with a negative impact
    firePosDa=xarray.concat(
        (posEvents.sel(index=firePos), negEvents.sel(index=fireNeg)
        ),
        'index')
    
    # a negative impact is a negative event with a positive impact, or a positive event with a negative impact
    fireNegDa=xarray.concat(
        (negEvents.sel(index=firePos),posEvents.sel(index=fireNeg)
        ),
        'index')
    
    return firePosDa, fireNegDa    

In [7]:
def compound(fireDa):
    """
    
    
    For now, assuming there are only three indeces being assessed, more should be possible, but not in scope
    
    """
    
    #How many events were there in each year?
    fireCount=fireDa.sum(dim='index')
    fireCount=fireCount.assign_coords(index='count')
    fireCount=fireCount.expand_dims('index')
    fireDa = xarray.concat([fireDa, fireCount],'index')

    #Which indices are we using?
    indexNames=firePos.index.values

    ensoIndex=(set(_index.enso).intersection(indexNames))
    iodIndex=(set(_index.iod).intersection(indexNames))
    samIndex=(set(_index.sam).intersection(indexNames))

    indexNames=[*ensoIndex, *iodIndex, *samIndex]
    
    #For now, limiting ourselves to one for each driver
    if len(ensoIndex)!=1:
        raise Error('number of enso indeces is not 1')
    elif len(iodIndex)!=1:
        raise Error('number of iod indeces is not 1')
    elif len(samIndex)!=1:
        raise Error('number of sam indeces is not 1')

    #Which years are there all three
    threeDa=(fireDa.sel(index='count')==3)
    threeDa['index']='all3'
    threeDa=threeDa.expand_dims('index')
    fireDa=xarray.concat([fireDa,threeDa], 'index')

    pairs=list()
    
    #Match each index with those further along the index list
    #(Nested for loops are probably bad juju)
    for i1 in range(0,len(indexNames)):
        for i2 in range(i1+1, len(indexNames)):
            #Its a compound of those two, if they both occur, and excluding if its a compound of all three
            tempDa=(
                fireDa.sel(index=indexNames[i1])*
                fireDa.sel(index=indexNames[i2])*
                (fireDa.sel(index='all3')==False)
            )
            tempDa=tempDa.assign_coords(index=indexNames[i1]+'+'+indexNames[i2])
            tempDa=tempDa.expand_dims('index')
            fireDa=xarray.concat([fireDa,tempDa], 'index', combine_attrs='drop_conflicts')
            
            pairs.append(indexNames[i1]+'+'+indexNames[i2])
            
    fireDa=fireDa.assign_attrs({**fireDa.attrs, 'pairs':pairs})
            
    return fireDa